In [243]:
                        #转换成ieee754标准
import numpy as np
np.set_printoptions(suppress = True, threshold = np.inf, precision = 15)
import bitstring, random 
def ieee754H(flt):
    b = bitstring.BitArray(float=flt, length=32)
    return b
def ieee754B(flt):
    aa = bitstring.BitArray(float=flt, length=32)
    aa = str(aa)
    aa = aa[2:]
    aa = bin(int(aa, 16))
    aa = aa[2:]
    aa = aa.rjust(32,'0')
    return aa
import struct
def hex2float(fp32):
    y=int(fp32,16)
    z=struct.unpack('<f',struct.pack('<I',y))
    # print(z[0])
    return z[0]


def fp16(number):
    
    float16 = np.float16(number)
    binary_representation = np.binary_repr(np.frombuffer(float16.tobytes(), dtype=np.uint16)[0], 16)
   
    return binary_representation


In [244]:
import numpy as np

def binary_str_to_fp16(binary_str):
    if len(binary_str) != 16:
        return "输入的二进制字符串长度必须为16位"
    
    # 分解为符号位、指数位和尾数位
    sign_bit = int(binary_str[0], 2)
    exp_bits = int(binary_str[1:6], 2)
    frac_bits = int(binary_str[6:], 2)
    
    # 根据IEEE 754的规则进行转换
    if exp_bits == 0:
        if frac_bits == 0:
            return -0.0 if sign_bit else 0.0
        else:
            exp = 2 ** -14  # 子规格数的指数部分
            frac = frac_bits / 2 ** 10
            return (-1) ** sign_bit * exp * frac
    elif exp_bits == 31:  # 0b11111
        if frac_bits == 0:
            return float('-inf') if sign_bit else float('inf')
        else:
            return float('nan')
    else:
        exp = 2 ** (exp_bits - 15)
        frac = 1 + frac_bits / 2 ** 10  # 加上隐藏位
        return (-1) ** sign_bit * exp * frac





In [245]:
def twos_complement(bin_str):
    # 如果输入为全0, 则直接返回原字符串
    if all(ch == '0' for ch in bin_str):
        return bin_str
    
    # 对字符串中的每个字符取反
    inverted_str = ''.join('1' if ch == '0' else '0' for ch in bin_str)
    
    # 将取反后的字符串转为整数，并加1
    inverted_num = int(inverted_str, 2) + 1
    
    # 将结果转回二进制字符串
    return bin(inverted_num)[2:].zfill(len(bin_str))

In [246]:
                        #exp max模块
def expmax_moudle(exp):
    expMax = exp[0]
    for i in range(64):
        if(expMax < exp[i]):
                expMax = exp[i]
    return expMax

In [247]:
                        #shift orchest模块
def shiftOrchest_moudle(w_manti_list):
    sewo = [[""] * 64 for _ in range(11)]
    for i in range(10,-1,-1):
        for j in range(64):
            sewo[10-i][j] = w_manti_list[j][i]
            #print('sewo[',i,'][',j,'] = manti[',j,'][',i,']')
    '''for k in range(24):
        print(sewo[k])'''
    #print(sewo[0])

    #print(type(sewo[0][0]))
    return sewo

In [248]:
                    #decoder模块
def decoder_moudle(sewo,a_manti,sign,i):
    shift_a_manti = [""] * 64
    int_shift_a_manti = [0]*64
    result_num = 0
    result_num_list = []
    result_str = [""] * 8
    #print('sewo[', i,'] = ',sewo)

    
    
    for k in range(64):     #把a_manti进行移位，变补码
        if sign[k] == 0:
            shift_a_manti[k] = a_manti[k][:i+1]
            shift_a_manti[k] = shift_a_manti[k].rjust(32,'0')
            int_shift_a_manti[k] = int(shift_a_manti[k],2)
        else:
            shift_a_manti[k] = a_manti[k][:i+1]
            shift_a_manti[k] = twos_complement(shift_a_manti[k])
            shift_a_manti[k] = shift_a_manti[k].rjust(32,'1')
            int_shift_a_manti[k] = int(shift_a_manti[k],2)
        ##print('shift_a_manti[',k,'] = ',bin(int_shift_a_manti[k])[2:])
    
    
    for b in range(8):      #第几个窗口
        result_num = 0
        for c in range(8):  #窗口里的哪个数
            #print(int(sewo[8 * b + c]) == 1)
            if int(sewo[8 * b + c]) == 1 :
                #print('int_shift_a_manti[',8 * b + c,' ] = ',bin(int_shift_a_manti[8 * b + c]))
                result_num = result_num + int_shift_a_manti[8 * b + c]
                #print('before result_num  ',8 * b + c,' = ',result_num)
                result_num = str(bin(result_num)[2:]).rjust(32,'0')
                result_num = result_num[-32:]
                result_num = int(result_num,2)
                #print('result_num [',8 * b + c,' ] = ',bin(result_num)[2:])
                ################################加完移除问题
            else:
                result_num = result_num + 0
        result_num_list.append(result_num)
    #print(result_num_list)


    return result_num_list

In [249]:
                        #adder tree模块
def adder_Tree_moudle(outnum):
    result = 0
    for a in range(8):
        for b in range(11):
            result = result + outnum[b][a]
    result = str(bin(result)[2:])
    result = result[-32:]
    result = int(result,2)
    #print('result = ',result)
    #print('result is ',twos_complement(str(bin(result)[2:])))
    return result

In [250]:
def post_process_moudle(result,emax):
    data_sign = str(bin(result)[2:])[0]
    #print('data_sign = ',data_sign)
    if(int(data_sign) == 1):
        result = twos_complement(str(bin(result)[2:]))
    cnt = 0
    offste = 0
    data_exp = 0
    for a in range(32):
        if(int(result[a]) == 0):
            cnt = cnt +1
        else:
            break
    #print('cnt = ',cnt)
    if(cnt < 21):
        result = result[cnt + 1:cnt + 11]
        data_manti = result
        offste = 21 - cnt
        data_exp = str(bin(emax - 15 + offste)[2:]).ljust(5,'0')
        #print('offset = ',offste)
        #print('data_manti = ',result)
        #print('data_exp = ',data_exp)
    else:
        result = result[cnt:].ljust(11,'0')
        offste = cnt - 11
        data_exp = emax - 15 - offste
    data = data_sign + str(data_exp) + data_manti
  
    return data

In [251]:
                        #输入预处理

import numpy as np

weight = np.array([
       -0.4604, 0.4248, -0.6665, 0.1401, -0.4995, -0.2224, -0.1085, -0.5459, -0.2146, 0.3069,
       0.7065, 0.2617, 0.05487, 0.7993, -0.6582, -0.05475, 0.834, -0.8525, -0.1818, -0.2959,
       -0.8022, 0.4094, 0.3965, -0.01563, 0.5391, -0.4849, 0.7568, -0.6973, 0.4822, 0.4717,
       0.06232, 0.4807, -0.5913, -0.6543, -0.2458, -0.6904, 0.8853, -0.2688, 0.1982, -0.2993,
       0.8262, -0.5098, -0.8281, -0.03482, -0.01611, 0.2676, 0.2617, -0.5835, -0.5732, -0.5093,
       -0.4849, -0.6758, 0.8813, -0.09155, 0.4248, -0.01929, -0.1879, 0.2546, -0.0159, -0.5923,
       -0.6055, -0.04816, 0.2998, 0.7075
        ])

active = np.array([
       -0.2861, -0.5542, 0.01114, 0.6328, 0.7617, 0.07062, -0.5518, 0.6104, 0.6172, 0.5039,
       -0.7686, -0.1838, 0.5962, -0.6021, 0.2278, 0.8047, -0.7437, -0.4165, -0.752, 0.8721,
       0.3545, -0.3384, -0.8286, 0.2339, 0.2278, -0.4539, -0.8872, 0.145, 0.3823, 0.4141,
       -0.5288, -0.4692, -0.8848, 0.5303, -0.2493, -0.6694, -0.3113, -0.3293, 0.5938, -0.4451,
       -0.6602, 0.4717, -0.8701, -0.5332, -0.2927, 0.1187, -0.3687, 0.5581, 0.7197, 0.6768,
       0.1433, 0.3555, -0.2549, 0.7397, 0.3572, -0.209, 0.3271, 0.8486, 0.5073, -0.3855,
       0.8335, 0.4854, 0.1792, 0.7612
        ])

#print('weight = ',weight[0])
#print('active = ',active)

                        #软件模拟结果   
mac_result_software = 0
for a in range(64):
    #print(a)
    multiply_mid = weight[a] * active[a]
    mac_result_software = multiply_mid + mac_result_software
print('the mac result is ',mac_result_software)

                        #预处理           

exp = [0] *64
weight_ieee754 = [""] * 64
active_ieee754 = [""] * 64

                        #处理exp 符号
sign = [""] * 64
for b in range(64):     
        weight_ieee754[b] = str(fp16(weight[b])) 
        active_ieee754[b] = str(fp16(active[b]))
        e_w = weight_ieee754[b][1:5]
        e_a = active_ieee754[b][1:5]
        exp[b] = int(e_w, 2) + int(e_a, 2)
        sign[b] = int(weight_ieee754[b][0]) ^ int(active_ieee754[b][0])

'''for z in range(63):
      print(exp[z])'''


                        #选出emax

expMax = expmax_moudle(exp)     
#print('expmax is ',expMax)
#print(type(expMax))


w_manti_list = []
a_manti_list = []
for c in range(64):
        
        w_manti = "1" + weight_ieee754[c][6:]
        w_manti = w_manti[:11-(int(expMax-exp[c]))]
        w_manti = w_manti.rjust(11,'0')
        w_manti_list.append(w_manti)
        

        a_manti = "1" + active_ieee754[c][6:]
        a_manti_list.append(a_manti)
'''''for z in range(64):
       print('a_manti_list[',z,'] = ',a_manti_list[z])'''''

sewo = shiftOrchest_moudle(w_manti_list)
#for z in range(24):
#print(len(w_manti_list[0]))
'''for z in range(24):
       print(w_manti_list[z])'''

outnum = []
for d in range(11):
       outnum.append(decoder_moudle(sewo[d],a_manti_list,sign,d))
#print(outnum)
'''for d in range(24):
       outnum[d] = decoder_moudle(sewo[d],a_manti_list,sign)'''
'''for z in range(24):
       print(bin(outnum[z])[2:])'''
result = adder_Tree_moudle(outnum)

data = post_process_moudle(result,expMax)
#print('binnary_data = ',data)

# print('data = ',data)


result = binary_str_to_fp16(data)
print("the software result is:", result)
print('end')


the mac result is  -3.7870622800000002
the software result is: -3.869140625
end
